In [129]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [130]:
feature_id = 12
CONVERTED_DATABASE_NAME = '/home/ubuntu/UPS2_DIA_180907/UPS2_DIA_180907.sqlite'
SUMMED_REGIONS_DATABASE = '/home/ubuntu/UPS2_DIA_180907/UPS2_DIA_180907-features-{}-{}.sqlite'.format(feature_id, feature_id)

In [131]:
# get the feature's base peak id
db_conn = sqlite3.connect(SUMMED_REGIONS_DATABASE)
feature_list_df = pd.read_sql_query("select * from feature_list where feature_id={}".format(feature_id), db_conn)
db_conn.close()
if len(feature_list_df) > 0:
    base_peak_id = feature_list_df.iloc[0].base_peak_id.astype(int)
else:
    print("Empty feature list.")

DatabaseError: Execution failed on sql 'select * from feature_list where feature_id=12': no such table: feature_list

In [ ]:
base_peak_id

In [ ]:
db_conn = sqlite3.connect(SUMMED_REGIONS_DATABASE)
feature_isotopes_df = pd.read_sql_query("select * from feature_isotopes where feature_id={}".format(feature_id), db_conn)
db_conn.close()

In [ ]:
feature_isotopes_df

In [ ]:
fig, ax = plt.subplots()
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
ax.stem(feature_isotopes_df.mz_centroid, feature_isotopes_df.summed_intensity)
plt.xlabel('m/z')
plt.ylabel('intensity')
plt.margins(0.06)
plt.title('Resolved feature {}'.format(feature_id))
plt.show()